In [1]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature

rsa → lets you generate RSA keys.

padding → defines how messages are padded before signing/verifying (RSA needs padding to be secure).

hashes → provides secure hash functions (here, SHA-256).

default_backend() → selects the cryptography backend (implementation details).

InvalidSignature → exception raised when verification fails.

In [6]:
# --- Generate RSA key pair ---
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()

Generates a private RSA key (2048 bits, secure).

public_exponent=65537 is the standard choice for RSA (fast + secure).

From the private key, we derive the public key.

In [14]:
# --- Original message ---
message = b"This is a secure message from Alice."

b"..." means it’s stored as bytes, not a Python string.

This is important because cryptographic operations work on byte sequences.

In [ ]:
# --- Sign the message ---
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

private_key.sign(...) → creates a digital signature.

PSS (Probabilistic Signature Scheme) → secure padding scheme for RSA signatures.

MGF1 (Mask Generation Function) with SHA-256 is used inside PSS.

salt_length=PSS.MAX_LENGTH → adds randomness to prevent forgery attacks.

hashes.SHA256() → the message is hashed with SHA-256 before signing (signing the hash, not the raw message).

👉 The result (signature) is a large byte string (usually same size as RSA modulus, here 256 bytes).

In [12]:
print("Signature: ", signature)
# Print signature in hexadecimal
print("Signature (hex):", signature.hex())
# Print signature in bits
print("Signature (bits):", ''.join(f"{byte:08b}" for byte in signature))

Signature:  b"Y\x05q\xf3\xff\xb1\xf9\x0b\xb8c\xde\xb1\x05\xe8(\xc9\x98\x8e9\x7f\x9d\x08U\x14\xe6Ol\xd4\x9ao}\xfc)\xba\x0c\xa9\xd9\x913F\xda\xb2\xeb3\x90`\xd4\x8e\x03\x88\xf1\x90\xebNS\xc4.\x9d\x10\xfe \x80\xec\x1f\xa5\xb9#\xf8\xbe#\x15\x9b\x8aL\x11\xdec\xd5m{\x91\xa2\xcb]qv4\x17\xb0\x82\x11\xaf'H\xaa\xcdq\x02q\xf4T\xfe\x91\xda3p\xc12s\x8dF2\xd0\xae_\x87\x198\x8d\xe2\x92m\xd8Q\x0e\xd6\x96\x1a\xfc\x8c^\x03\xfa\t\x1c\xa38k\xfa\xb7M\xa5\xdb\xc7\xfb\x86\xc3\x14O\xb4\n\x8a\xfb\x88\xe7\x8c\xe3'\x9d\xa3\xb3s\x1e>\xec\xc3Y]\xfc\x02\\\xfb\xd0p\x1f\xb1\x84\x8f\xf7eQp\x18\xdf\xa3\xa9\x7f\x823\xac0r\xde\x9b\r\xb6\xea\x08\x148\x81\xb4@&\xc8\x9c\x18\xe1\\q\x89c\x07\xa1s\xf5?x\xdf\x7f/\xfd]\xcf\x13\xaa=\xb3\xde\x83'\xcd\x13p\xbf\x12h\xf1@\xdd\xebev\xfe\xb8\xcf\xa1Jv\t\x00\x8a[p\x01I"
Signature (hex): 590571f3ffb1f90bb863deb105e828c9988e397f9d085514e64f6cd49a6f7dfc29ba0ca9d9913346dab2eb339060d48e0388f190eb4e53c42e9d10fe2080ec1fa5b923f8be23159b8a4c11de63d56d7b91a2cb5d71763417b08211af2748aacd710271f454fe

In [13]:
# --- Function to verify and report ---
def verify_signature(msg, sig, label="Original"):
    try:
        public_key.verify(
            sig,
            msg,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print(f"✅ {label}: Signature is valid.")
    except InvalidSignature:
        print(f"❌ {label}: Signature is invalid!")

# --- 1. Original (valid) ---
verify_signature(message, signature, "Original")

# --- 2. Tampered message ---
tampered_message = b"This is a secure message from Eve."
verify_signature(tampered_message, signature, "Tampered Message")

# --- 3. Tampered signature ---
tampered_signature = signature[:-1] + b'\x00'  # corrupt last byte
verify_signature(message, tampered_signature, "Tampered Signature")



✅ Original: Signature is valid.
❌ Tampered Message: Signature is invalid!
❌ Tampered Signature: Signature is invalid!


public_key.verify(...) checks that:

The sig was produced with the corresponding private key.

The msg has not been altered.

The padding and hash match what was used at signing.

If verification fails, it raises InvalidSignature.